# Uncertainties in Greenland mass loss due to uncertainties in accounting

### Group Project International Summer School in Glaciology, McCarthy, June 7-17 2024

## **How much mass is the Greenland Ice Sheet losing?**

Published numbers vary not only due to the use of different data sets and methods, but also due to use of different ice sheet and basin outlines. Sometimes, glaciers and ice caps disconnected from the main ice sheet are included, sometimes they are not. This makes comparison challenging, and a community consensus is needed. In this project, we will explore different methods for basin delineation.

<img src="gris_basins_tweet.png" alt="Tweet by Hester Jiskoot on the plethora of Greenland basins" style="width: 500px;"/>

## Method 1: Delineation using flowlines

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gp
from shapely import Point
import pylab as plt

from glacier_flow_tools.interpolation import velocity
from glacier_flow_tools.pathlines import (
    compute_pathline,
    series_to_pathline_geopandas_dataframe,
)
from glacier_flow_tools.utils import register_colormaps
register_colormaps()

In [ ]:
hmin = 1
hmax = 25.0
tol = 1
start_time = 0
end_time = 5_000

ds = xr.open_dataset("data/its_live_jak.nc")
Vx = np.squeeze(ds["vx"].to_numpy())
Vy = np.squeeze(ds["vy"].to_numpy())

# Reverse for backward pathlines
Vx = -Vx
Vy = -Vy

x = ds["x"].to_numpy()
y = ds["y"].to_numpy()

geom = [Point(-188_000, -2_255_000), Point(-182_000, -2_290_000)]
starting_points_df = gp.GeoDataFrame(geometry=geom, crs="EPSG:3413")

pathlines = []
for _, df in starting_points_df.iterrows():
    pathline = compute_pathline(
        [*df.geometry.coords[0]],
        velocity,
        f_args=(Vx, Vy, x, y),
        hmin=hmin,
        hmax=hmax,
        tol=tol,
        start_time=start_time,
        end_time=end_time,
        notebook=True,
        progress=True,
    )
    pathlines.append(pathline)


In [ ]:
# Combine the pathlines into a GeoPandas DataFrame

result = pd.concat(        [
    series_to_pathline_geopandas_dataframe(s.drop("geometry", errors="ignore"), pathlines[k])
    for k, s in starting_points_df.iterrows()
]
                  ).reset_index(drop=True)


In [ ]:
ratio = (y.max() - y.min()) / (x.max() - x.min())
fig, ax = plt.subplots(1, figsize=(12, 12 * ratio))
ds["v"].plot(ax=ax, cmap="speed_colorblind", vmin=10, vmax=1500)
result.plot(markersize=25,
            color="w",
            legend=False,
            missing_kwds={},
            ax=ax,
            edgecolors="k",
            linewidths=0.5,
)

## Tasks

- Visualize different drainage basins
- Calculate pathlines and find a way to close them, making a polygon
- Explore role of uncertainties in ITS_LIVE
- Smooth DEM and use Drainage basin calculator